In [1]:
import socket
import threading
import platform
import ipaddress
import subprocess
from queue import Queue
from datetime import datetime

print_lock = threading.Lock()
queue = Queue()

# أشهر البروتوكولات وبورتاتها
COMMON_PORTS = {
    21: ("FTP", "tcp"),
    22: ("SSH", "tcp"),
    23: ("Telnet", "tcp"),
    25: ("SMTP", "tcp"),
    53: ("DNS", "udp"),
    80: ("HTTP", "tcp"),
    110: ("POP3", "tcp"),
    143: ("IMAP", "tcp"),
    389: ("LDAP", "tcp"),
    443: ("HTTPS", "tcp"),
    445: ("SMB", "tcp"),
    161: ("SNMP", "udp"),
    3389: ("RDP", "tcp")
}

def grab_banner(ip, port):
    try:
        s = socket.socket()
        s.settimeout(2)
        s.connect((ip, port))
        banner = s.recv(1024).decode(errors="ignore").strip()
        s.close()
        return banner if banner else "No banner"
    except:
        return "No banner"

def detect_os(ip):
    try:
        param = '-n' if platform.system().lower() == 'windows' else '-c'
        output = subprocess.check_output(['ping', param, '1', ip], stderr=subprocess.STDOUT).decode()
        if 'ttl=' in output.lower():
            ttl = int(output.lower().split('ttl=')[1].split()[0])
            if ttl <= 64:
                return "Linux/Unix (approx)"
            elif ttl <= 128:
                return "Windows (approx)"
            else:
                return "Unknown OS (high TTL)"
        else:
            return "Could not detect OS (no TTL found)"
    except Exception as e:
        return f"Error detecting OS: {e}"

def scan_tcp(target, port, results):
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.settimeout(1)
        s.connect((target, port))
        try:
            service = socket.getservbyport(port)
        except:
            service = "Unknown"
        banner = grab_banner(target, port)
        with print_lock:
            print(f"[+] TCP Port {port:<5} | Service: {service:<15} | Banner: {banner}")
        results.append((port, 'TCP', service, banner))
        s.close()
    except:
        pass

def scan_udp(target, port, results):
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.settimeout(1)
        s.sendto(b'', (target, port))
        s.recvfrom(1024)
        with print_lock:
            print(f"[+] UDP Port {port:<5} is open (maybe)")
        results.append((port, 'UDP', 'Unknown', 'No banner'))
        s.close()
    except socket.timeout:
        with print_lock:
            print(f"[+] UDP Port {port:<5} is open|filtered")
        results.append((port, 'UDP', 'Unknown', 'Filtered/No response'))
    except:
        pass

def worker(protocol, target, results):
    while not queue.empty():
        port = queue.get()
        if protocol == "tcp":
            scan_tcp(target, port, results)
        elif protocol == "udp":
            scan_udp(target, port, results)
        elif protocol == "all":
            proto_info = COMMON_PORTS.get(port, None)
            if proto_info:
                proto_name = proto_info[1]
                if proto_name == "tcp":
                    scan_tcp(target, port, results)
                elif proto_name == "udp":
                    scan_udp(target, port, results)
        queue.task_done()

def get_targets(ip_or_range):
    try:
        return [str(ip) for ip in ipaddress.IPv4Network(ip_or_range, strict=False)]
    except:
        return [ip_or_range]

def save_report(target_ip, results, os_info):
    filename = f"scan_report_{target_ip.replace('.', '_')}.txt"
    with open(filename, "w") as f:
        f.write(f"Scan Report for {target_ip}\n")
        f.write(f"OS Detection: {os_info}\n\n")
        for port, proto, service, banner in results:
            f.write(f"{proto} {port}/{'tcp' if proto == 'TCP' else 'udp'} | {service} | Banner: {banner}\n")
    print(f"\n📁 Report saved to {filename}")

def main():
    print("\n💥 Python Port Scanner + Service Detection + OS Fingerprinting 💥\n")

    ip_input = input("🎯 Enter IP, domain or IP range (e.g. 192.168.1.0/30): ").strip()
    protocol = input("📡 Protocol to scan (tcp/udp/all): ").strip().lower()

    if protocol not in ["tcp", "udp", "all"]:
        print("❌ Invalid protocol. Use tcp, udp, or all.")
        return

    if protocol != "all":
        ports_input = input("🔢 Enter ports (e.g. 20-80 or 22,80,443): ").strip()
        ports = []
        try:
            if "-" in ports_input:
                start, end = map(int, ports_input.split("-"))
                ports = list(range(start, end + 1))
            else:
                ports = list(map(int, ports_input.split(",")))
        except:
            print("❌ Invalid port format.")
            return
    else:
        ports = list(COMMON_PORTS.keys())

    targets = get_targets(ip_input)

    for target in targets:
        try:
            ip = socket.gethostbyname(target)
        except socket.gaierror:
            print(f"[!] Could not resolve: {target}")
            continue

        print(f"\n🎯 Target: {target} ({ip})")
        print(f"📌 Protocol: {protocol.upper()}")
        print(f"📍 Port Range: {ports[0]} - {ports[-1]}")
        print("🧠 Detecting OS...")
        os_info = detect_os(ip)
        print(f"   → OS: {os_info}\n")

        results = []
        start_time = datetime.now()

        for port in ports:
            queue.put(port)

        for _ in range(100):  # threads
            t = threading.Thread(target=worker, args=(protocol, ip, results))
            t.daemon = True
            t.start()

        queue.join()

        duration = datetime.now() - start_time
        print(f"\n✅ Scan complete in {duration.total_seconds():.2f} seconds.")
        save_report(ip, results, os_info)

if __name__ == "__main__":
    main()



💥 Python Port Scanner + Service Detection + OS Fingerprinting 💥


🎯 Target: rayashop.com (172.67.68.242)
📌 Protocol: ALL
📍 Port Range: 21 - 3389
🧠 Detecting OS...
   → OS: Linux/Unix (approx)

[+] UDP Port 53    is open|filtered
[+] UDP Port 161   is open|filtered
[+] TCP Port 80    | Service: http            | Banner: No banner
[+] TCP Port 443   | Service: https           | Banner: No banner

✅ Scan complete in 2.14 seconds.

📁 Report saved to scan_report_172_67_68_242.txt


In [ ]:
import socket
import threading
import platform
import subprocess
import ipaddress
import json
from queue import Queue
from datetime import datetime



print_lock = threading.Lock()
queue = Queue()

# أشهر البروتوكولات وبورتاتها
COMMON_PORTS = {
    21: ("FTP", "tcp"),
    22: ("SSH", "tcp"),
    23: ("Telnet", "tcp"),
    25: ("SMTP", "tcp"),
    53: ("DNS", "udp"),
    80: ("HTTP", "tcp"),
    110: ("POP3", "tcp"),
    143: ("IMAP", "tcp"),
    389: ("LDAP", "tcp"),
    443: ("HTTPS", "tcp"),
    445: ("SMB", "tcp"),
    161: ("SNMP", "udp"),
    3389: ("RDP", "tcp")
}

def grab_banner(ip, port):
    try:
        s = socket.socket()
        s.settimeout(2)
        s.connect((ip, port))
        banner = s.recv(1024).decode(errors="ignore").strip()
        s.close()
        return banner if banner else "No banner"
    except:
        return "No banner"

def detect_os(ip):
    try:
        param = '-n' if platform.system().lower() == 'windows' else '-c'
        output = subprocess.check_output(['ping', param, '1', ip], stderr=subprocess.STDOUT).decode()
        if 'ttl=' in output.lower():
            ttl = int(output.lower().split('ttl=')[1].split()[0])
            if ttl <= 64:
                return "Linux/Unix (approx)"
            elif ttl <= 128:
                return "Windows (approx)"
            else:
                return "Unknown OS (high TTL)"
        else:
            return "Could not detect OS (no TTL found)"
    except Exception as e:
        return f"Error detecting OS: {e}"

def scan_tcp(target, port, results):
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.settimeout(1)
        s.connect((target, port))
        try:
            service = socket.getservbyport(port)
        except:
            service = "Unknown"
        banner = grab_banner(target, port)
        with print_lock:
            print(f"[+] TCP Port {port:<5} | Service: {service:<15} | Banner: {banner}")
        results.append((port, 'TCP', service, banner))
        s.close()
    except:
        pass

def scan_udp(target, port, results):
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.settimeout(1)
        s.sendto(b'', (target, port))
        s.recvfrom(1024)
        with print_lock:
            print(f"[+] UDP Port {port:<5} is open (maybe)")
        results.append((port, 'UDP', 'Unknown', 'No banner'))
        s.close()
    except socket.timeout:
        with print_lock:
            print(f"[+] UDP Port {port:<5} is open|filtered")
        results.append((port, 'UDP', 'Unknown', 'Filtered/No response'))
    except:
        pass

def worker(protocol, target, results):
    while not queue.empty():
        port = queue.get()
        if protocol == "tcp":
            scan_tcp(target, port, results)
        elif protocol == "udp":
            scan_udp(target, port, results)
        elif protocol == "all":
            proto_info = COMMON_PORTS.get(port)
            if proto_info:
                proto_name = proto_info[1]
                if proto_name == "tcp":
                    scan_tcp(target, port, results)
                elif proto_name == "udp":
                    scan_udp(target, port, results)
        queue.task_done()

def save_report(domain, ip, results, os_info):
    filename = f"scan_report_{domain.replace('.', '_')}.txt"
    with open(filename, "w") as f:
        f.write(f"Scan Report for {domain} ({ip})\n")
        f.write(f"OS Detection: {os_info}\n\n")
        for port, proto, service, banner in results:
            f.write(f"{proto} {port} | {service} | Banner: {banner}\n")
    print(f"\n📁 Report saved to {filename}")

def run_scan_on_target(domain, ip, protocol, ports):
    print(f"\n🎯 Target: {domain} ({ip})")
    print(f"📌 Protocol: {protocol.upper()}")
    print(f"📍 Ports: {', '.join(map(str, ports))}")
    print("🧠 Detecting OS...")
    os_info = detect_os(ip)
    print(f"   → OS: {os_info}\n")

    results = []
    start_time = datetime.now()

    for port in ports:
        queue.put(port)

    for _ in range(100):
        t = threading.Thread(target=worker, args=(protocol, ip, results))
        t.daemon = True
        t.start()

    queue.join()
    duration = datetime.now() - start_time

    print(f"\n✅ Scan complete for {domain} in {duration.total_seconds():.2f} seconds.")
    save_report(domain, ip, results, os_info)

def main():
    print("\n💥 Python Port Scanner + Subdomain List Support 💥")
    json_path = input("📁 Enter path to JSON file with subdomains: ").strip()
    protocol = input("📡 Protocol to scan (tcp/udp/all): ").strip().lower()

    if protocol not in ["tcp", "udp", "all"]:
        print("❌ Invalid protocol. Use tcp, udp, or all.")
        return

    if protocol != "all":
        ports_input = input("🔢 Enter ports (e.g. 20-80 or 22,80,443): ").strip()
        ports = []
        try:
            if "-" in ports_input:
                start, end = map(int, ports_input.split("-"))
                ports = list(range(start, end + 1))
            else:
                ports = list(map(int, ports_input.split(",")))
        except:
            print("❌ Invalid port format.")
            return
    else:
        ports = list(COMMON_PORTS.keys())

    # Read subdomain JSON
    try:
        with open(json_path, "r") as f:
            subdomains = json.load(f)
    except Exception as e:
        print(f"❌ Failed to load JSON: {e}")
        return

    for entry in subdomains:
        domain = entry.get("domain")
        ip = entry.get("ip")
        if not ip or not domain:
            print(f"[!] Skipping invalid entry: {entry}")
            continue
        run_scan_on_target(domain, ip, protocol, ports)

if __name__ == "__main__":
    main()



💥 Python Port Scanner + Subdomain List Support 💥
